In [1]:
# loading data into environment 

import pandas as pd

df = pd.read_csv('customer_segmentation.csv')
df.head()

,order_id,order_date,region,product,category,sales,cost,quantity,customer_id
0,5136,2023-01-09,East,Laptop,Electronics,56184,47506,1,CUST021
1,5033,2023-01-10,East,Table,Furniture,11604,10713,1,CUST007
2,5134,2023-01-11,East,Table,Furniture,10078,7910,4,CUST032
3,5223,2023-01-14,West,Mobile,Electronics,21974,14751,4,CUST042
4,5058,2023-01-16,West,Headphones,Electronics,848,1386,4,CUST054


In [2]:
# converting order-date data type because it is in text 
# we convert it using to_datetime() function in pandas

df['order_date'] = pd.to_datetime(df['order_date'])

In [3]:
df.head()

,order_id,order_date,region,product,category,sales,cost,quantity,customer_id
0,5136,2023-01-09,East,Laptop,Electronics,56184,47506,1,CUST021
1,5033,2023-01-10,East,Table,Furniture,11604,10713,1,CUST007
2,5134,2023-01-11,East,Table,Furniture,10078,7910,4,CUST032
3,5223,2023-01-14,West,Mobile,Electronics,21974,14751,4,CUST042
4,5058,2023-01-16,West,Headphones,Electronics,848,1386,4,CUST054


In [4]:
# calculating the last transaction for warehouse added one to avoid last transaction

snapshot_date = df['order_date'].max()+pd.Timedelta(days = 1)
print(snapshot_date)

2024-12-29 00:00:00


In [5]:
# calculating recency,frequency,monetary from table 
# recency -- most recently visited
# frequency -- no of times visited
# monetary -- total amount spent

rfm = df.groupby('customer_id').agg({
                 'order_date': lambda x:(snapshot_date-x.max()).days,
                  'order_id': 'count',
                'sales' : 'sum'}).reset_index()

# renaming column names

rfm.columns = ['customer_id', 'recency', 'frequency', 'monetary']

rfm.head()

,customer_id,recency,frequency,monetary
0,CUST001,232,2,14291
1,CUST002,98,4,38984
2,CUST003,571,3,65877
3,CUST004,171,5,138390
4,CUST005,18,4,131396


In [6]:
# calculating scores according to their value

rfm['R_Score'] = pd.qcut(rfm['recency'],4,labels = [4,3,2,1])
rfm['F_Score'] = pd.qcut(rfm['frequency'].rank(method ='first'),4,labels = [1,2,3,4])
rfm['M_Score'] = pd.qcut(rfm['monetary'],4,labels = [1,2,3,4])

rfm['RFM_score'] = rfm[['R_Score','F_Score','M_Score']].astype(int).sum(axis=1)

rfm.head()                                           

,customer_id,recency,frequency,monetary,R_Score,F_Score,M_Score,RFM_score
0,CUST001,232,2,14291,1,1,1,3
1,CUST002,98,4,38984,3,2,2,7
2,CUST003,571,3,65877,1,1,3,5
3,CUST004,171,5,138390,2,3,4,9
4,CUST005,18,4,131396,4,2,4,10


In [7]:
#defining segment names

def segment_customer(score):
    if score >= 10:
        return "High Value"
    elif score >= 7:
        return "Medium Value"
    else:
        return "Low Value"

rfm['Segment'] = rfm['RFM_score'].apply(segment_customer)

rfm['Segment'].value_counts()

Segment
Low Value       26
Medium Value    17
High Value      17
Name: count, dtype: int64

In [8]:
# we are converting rfm table to csv file for better understanding purpose

rfm.to_csv("Cust_level.csv", index=False)